In [ ]:
import os
from dotenv import load_dotenv
import pathlib
import textwrap

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY)

In [1]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader
from collections import Counter

loader = DirectoryLoader('.', glob="*.txt")
documents = loader.load()

## 랭체인

In [2]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.chat_models import  ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import  TextLoader, DirectoryLoader
from collections import  Counter


loader = DirectoryLoader("./data",glob="*.txt",loader_cls=TextLoader)
documents = loader.load()
print("문서의 개수",len(documents))


문서의 개수 57


청크나누기

In [3]:
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)
print("텍스트의 개수",len(chunks))
print(chunks[0])

텍스트의 개수 64
page_content='정책제목: 청년과 어르신 주거공유(한지붕세대공감)\n서울시청 주택정책과에서는 해당 지역 대학생들의 주거 마련을 위해 할머니와 할아버지들이 남는 방을 학생에게 저렴한 가격에 빌려주는 정책을 운영하고 있습니다. 이 정책은 대학가나 청년들이 많이 사는 지역에서 진행되며, 어르신들은 시세보다 저렴한 값으로 방을 빌려주고, 학생들은 무보증금으로 학교 근처에 근접한 주거공간을 제공받습니다.\n\n어르신들은 주택을 소유한 60세 이상 어르신으로서 신청할 수 있으며, 대학생은 서울시 소재 대학(원) 재학생 및 휴학생으로 신청 자격을 갖추고 있어야 합니다. 학력이나 전공에 대한 제한은 없으며, 취업을 하지 않은 미취업자들이 참여할 수 있습니다.\n\n이 정책은 2023년 1월 1일부터 2023년 12월 31일까지 운영되며, 신청은 서울주거포털 사이트(https://housing.seoul.go.kr)에서 온라인으로 진행됩니다. 운영 기관은 주관기관과 동일하며, 자세한 신청 방법 및 제출서류는 공고문을 참조하시기 바랍니다.\n\n자세한 사항은 서울주거포털 사이트(https://housing.seoul.go.kr/)에서 확인하실 수 있습니다.' metadata={'source': 'data/29.txt'}


크로마 db, 임베딩

In [9]:
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks,embedding=embeddings)

retriever = vectordb.as_retriever()
docs = retriever.get_relevant_documents("신혼 부부를 위한 정책이 있어?")
print('유사 문서 개수 :', len(docs))
print('--' * 20)
print('첫번째 유사 문서 :', docs[0])
print('--' * 20)
print('각 유사 문서의 문서 출처 :')
for doc in docs:
    print(doc.metadata["source"])

유사 문서 개수 : 4
----------------------------------------
첫번째 유사 문서 : page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.' metadata={'source': 'data/23.txt'}
----------------------------------------
각 유사 문서의 문서 출처 :
data/23.txt
data/23.txt
data/23.txt
data/23.txt


In [12]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(api_key=OPENAI_API_KEY,model_name="gpt-4o", temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

In [13]:
input_text = "대출과 관련된 정책이 궁금합니다"
chatbot_response = qa_chain(input_text)
print(chatbot_response)


/Users/seungwoo/anaconda3/envs/pyserverprog/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': '대출과 관련된 정책이 궁금합니다', 'result': '서울시에서 제공하는 학자금대출 신용회복 지원사업에 대해 안내드리겠습니다. 이 정책은 서울특별시 미래청년기획단이 주최하는 금융 정책으로, 학자금 대출로 인해 신용이 떨어져 어려움을 겪고 있는 청년층을 위한 신용회복 지원을 목적으로 합니다.\n\n### 주요 내용:\n1. **지원 내용**:\n   - 분할상환약정 체결 지원\n   - 초입금 제공\n   - 신용유의자 등록 해제\n\n2. **지원 대상**:\n   - 서울에 거주하는 19세부터 39세까지의 청년\n   - 학자금 대출로 인해 신용유의자인 청년\n   - 약 200여명 지원\n   - 2018년부터 2022년에 지원을 받은 사람은 2023년 지원 대상에서 제외\n\n3. **신청 방법**:\n   - 서울청년포털(youth.seoul.go.kr)을 통해 신청\n   - 심사 및 발표는 매월 1~2회 선정되며, 신청인원에 따라 주기가 변동될 수 있음\n\n4. **운영 기관**:\n   - 서울시 미래청년기획단\n\n### 참고 사이트:\n- [신용회복 신청 안내](https://youth.seoul.go.kr/site/main/board/notice/27789?baCategory1=basic&baCommSelec=true)\n- [신청 사이트](https://youth.seoul.go.kr/site/main/applyLoanCredit/step1)\n\n자세한 내용은 위의 링크를 통해 확인하실 수 있습니다. 추가적인 문의사항이 있으시면 관련 사이트를 참고하시거나 서울시 미래청년기획단에 문의해 주세요.', 'source_documents': [Document(page_content='정책내용: 서울시 학자금대출 신용회복 지원사업\n서울특별시 미래청년기획단이 주최하는 금융 정책으로, 학자금 대출로 인해 신용이 떨어져 어려움을 겪고 있는 청년층을 위한 신용회복 지원입니다. 이 정책은 분할상환약정 체결을 지원하고 초입금을 제공함으

In [14]:
def get_chatbot_response(chatbot_response):
    print(chatbot_response['result'].strip())
    print('\n문서 출처:')
    for source in chatbot_response["source_documents"]:
        print(source.metadata['source'])

In [15]:
input_text = "신혼 부부의 신혼집 마련을 위한 정책이 있을까?"
chatbot_response = qa_chain(input_text)
get_chatbot_response(chatbot_response)

네, 서울시에서는 신혼 부부의 신혼집 마련을 위한 정책을 운영하고 있습니다. 이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 마련되었습니다. 

이 정책의 주요 내용은 다음과 같습니다:

- **대상 가구**: 총 8,000가구로 제한
- **지원 기간**: 2023년 1월 1일부터 2023년 12월 31일까지
- **신청 자격**:
  - 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정인 자
  - 혼인신고일 기준으로 7년 이내의 신혼부부 또는 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부
  - 부부의 합산 연소득이 9천 7백만원 이하
  - 본인 및 배우자가 무주택자
  - 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자

- **신청 방법**: 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청
- **필요 서류**: 주민등록등본, 가족관계증명서, 혼인관계증명서, 임대차계약서

이 정책은 서울시청 주택정책과에서 운영하며, 자세한 사항은 서울주거포털(https://housing.seoul.go.kr)에서 확인할 수 있습니다.

문서 출처:
data/23.txt
data/23.txt
data/23.txt
data/23.txt


In [1]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader
from collections import Counter

import gradio as gr
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
loader = DirectoryLoader('./data', glob="*.txt")
documents = loader.load()

chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks,
                                 embedding=embeddings)
retriever = vectordb.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(api_key=OPENAI_API_KEY,model_name="gpt-4o", temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

def get_chatbot_response(chatbot_response):
    print(chatbot_response['result'].strip())
    print('\n문서 출처:')
    for source in chatbot_response["source_documents"]:
        print(source.metadata['source'])
        
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="사회복지 메뉴얼 도움 봇")
    msg = gr.Textbox(label="질문해주세요!")
    clear = gr.Button("Clear")
    
    def response(message, chat_history=[]):
        result = qa_chain(message)
        bot_message = result['result']
        bot_message += '\n#source :'
        for i,doc in enumerate(result['source_documents']):
            bot_message += '[' + str(i+1) +']'+ doc.metadata["source"] +'\n'
        chat_history.append((message, bot_message))
        return "", chat_history
    
    msg.submit(response, inputs=[msg,chatbot], outputs=[msg,chatbot])
    clear.click(lambda: None, None,chatbot,queue=False)
    
demo.launch()

/Users/seungwoo/anaconda3/envs/pyserverprog/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Expected IDs to be a non-empty list, got 0 IDs